In [1]:
import numpy
from src.data.datamanager import DataManager
from src.data.window import WindowSequence, KerasSequence
from src.data.dataset import MotionSense, Activity
from src.visual.plotter import Plotter, VecData
from src.network.gan import Gan, SimpleGan, SmoothingType, CGan, AdversarialAutoencoder, VaeGan
from src.network.generator import SimpleMlpGen
from src.network.discriminator import SimpleMlpDisc
from keras.optimizers import SGD, Adam, RMSprop
from pathlib import Path
import toml
datasets = toml.load("config.toml")["dataset"]
WISDM_PATH = Path(datasets["wisdm"])
MOTION_SENSE_PATH = Path(datasets["motion-sense"])
dataset = MotionSense(MOTION_SENSE_PATH)
datamanager = DataManager(dataset)
all_data=datamanager.read()
windows = datamanager.create_windows(set(Activity), 100, shuffle=True, seed=1, columns=['xaccel_norm', 'yaccel_norm', 'zaccel_norm', 'xrot_norm', 'yrot_norm', 'zrot_norm'])#, bypass_raw="6a3bf91cfb")
# windows = datamanager.create_windows(set(Activity), 100, shuffle=False, seed=1, columns=['xaccel_norm', 'yaccel_norm', 'zaccel_norm', 'xrot_norm', 'yrot_norm', 'zrot_norm', "activity"])#, bypass_raw="6a3bf91cfb")

Using TensorFlow backend.


In [2]:
import numpy
import pandas

n_latents = 200
num_classes = len(dataset.ACTIVITIES.values())

def generate_labels(n_samples: int, num_classes: int, onehot: bool) -> numpy.ndarray:
    if onehot:
        return numpy.eye(num_classes)[
            numpy.random.choice(num_classes, n_samples)
        ]
    return numpy.random.randint(0, num_classes, (n_samples, 1))

def smooth_positive_labels(y):
    return y - 0.3 + (random(y.shape) * 0.5)

def smooth_negative_labels(y):
    return y + random(y.shape) * 0.3


def noisy_labels(y, p_flip):
    # determine the number of labels to flip
    n_select = int(p_flip * y.shape[0])
    # choose labels to flip
    flip_ix = numpy.random.choice([i for i in range(y.shape[0])], size=n_select)
    # invert the labels in place
    y[flip_ix] = 1 - y[flip_ix]
    return y

# generate points in latent space as input for the generator
#Gaussian values with a mean close to zero and a 
#standard deviation close to 1, e.g. a standard Gaussian distribution.
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = numpy.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape((n_samples, latent_dim))
    return x_input

def create_ground_values(n_samples: int):
    return (
        numpy.clip(noisy_labels(smooth_positive_labels(numpy.ones((n_samples, 1))), 0.05), 0, 1.2),
        numpy.clip(noisy_labels(smooth_negative_labels(numpy.zeros((n_samples, 1))), 0.05), 0, 1.2),
    )

def generate_and_prepare_data(generator, n_samples_each=50):

    latents = generate_latent_points(n_latents, n_samples_each * num_classes)
    Y_gen = numpy.array([[i]*n_samples_each for i in range(num_classes)]).flatten()
    X_gen = generator.predict([latents, Y_gen])

    window_sample = datamanager.create_windows(
        set(Activity), 
        window_length, 
        shuffle=True,
        columns=['xaccel_norm', 'yaccel_norm', 'zaccel_norm', 'xrot_norm', 'yrot_norm', 'zrot_norm'])
    seq_sample = windows.to_keras_sequence(n_samples_each * num_classes)

    X_real = seq_sample[0][0]
    Y_real = seq_sample[0][1]
    
    return X_real, Y_real, X_gen, Y_gen

def generate_fooling_data(generator, discriminator):
    X_real, Y_real, X_gen, Y_gen = generate_and_prepare_data(generator)
    Y_pred_fakereal, _ = discriminator.predict(X_gen)
    Y_gen_fooled = numpy.delete(Y_gen, numpy.where(Y_pred_fakereal < 0.5), axis=0)
    X_gen_fooled = numpy.delete(X_gen, numpy.where(Y_pred_fakereal < 0.5), axis=0)
    return X_gen_fooled, Y_gen_fooled

def plot_fooling_sample(activity, generator, discriminator):
    X_gen_fooled, Y_gen_fooled = generate_fooling_data(generator, discriminator)
    
def plot_real(cls):
    real_data=all_data.loc[all_data["activity"] == cls].iloc[0:window_length]
    acc_cols = ('xaccel_norm', 'yaccel_norm', 'zaccel_norm')
    plotter = Plotter(VecData(real_data, acc_cols, 100), dataset.FREQUENCY)
    return plotter.make_line_plot(cls)

def plot_fake(generator, discriminator, cls):
    cls_code = sorted(dataset.ACTIVITIES.values()).index(cls)
    X_gen_fooled, Y_gen_fooled = generate_fooling_data(generator, discriminator)
    fake_data = X_gen_fooled[numpy.where(Y_gen_fooled == cls_code)][0]
    
    acc_cols = ('xaccel_norm', 'yaccel_norm', 'zaccel_norm')
    df = pandas.DataFrame(data=fake_data, columns=[*acc_cols, 'xrot_norm', 'yrot_norm', 'zrot_norm'])  
    plotter = Plotter(VecData(df, acc_cols, 100), dataset.FREQUENCY)
    return plotter.make_line_plot(cls)
        
    
def animate_real(cls):
    real_data=all_data.loc[all_data["activity"] == cls].iloc[0:window_length]
    acc_cols = ('xaccel_norm', 'yaccel_norm', 'zaccel_norm')
    plotter = Plotter(VecData(real_data, acc_cols, 1000), dataset.FREQUENCY)
    return plotter.make_2d_animations(cls)

def animate_fake(generator, discriminator, cls):
    cls_code = sorted(dataset.ACTIVITIES.values()).index(cls)
    X_gen_fooled, Y_gen_fooled = generate_fooling_data(generator, discriminator)
    fake_data = X_gen_fooled[numpy.where(Y_gen_fooled == cls_code)][0]
    
    acc_cols = ('xaccel_norm', 'yaccel_norm', 'zaccel_norm')
    df = pandas.DataFrame(data=fake_data, columns=[*acc_cols, 'xrot_norm', 'yrot_norm', 'zrot_norm'])  
    plotter = Plotter(VecData(df, acc_cols, 1000), dataset.FREQUENCY)
    return plotter.make_2d_animations(cls)

In [3]:
from keras.models import load_model

classifier = load_model('models/classifiers/cnn_classifier.h5')

/home/orcan/.local/share/virtualenvs/user-activity-generator-2M2ZtGjA/lib/python3.8/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
/home/orcan/.local/share/virtualenvs/user-activity-generator-2M2ZtGjA/lib/python3.8/site-packages/keras/engine/training_utils.py:811: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if isinstance(loss, collections.Mapping):
/home/orcan/.local/share/virtualenvs/user-activity-generator-2M2ZtGjA/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if not isinstance(values, collections.Sequence):


In [4]:
X = []
Y = []
Y_onehot = []
for x, y in windows.to_keras_sequence(1):
    x = x.squeeze(axis=0)
    y = y.squeeze(axis=0)
    Y_onehot.append(y)
    X.append(x)
    Y.append(y.argmax())
X = numpy.asarray(X)
Y = numpy.asarray(Y)
Y_onehot = numpy.asarray(Y_onehot)

boundary = round(len(X)*(70/100))
X_train, X_test = X[:boundary], X[boundary:]
Y_train, Y_test = Y[:boundary], Y[boundary:]
Y_train_onehot, Y_test_onehot = Y_onehot[:boundary], Y_onehot[boundary:]

s = X_train.shape
X_train_flat, X_test_flat = X_train.reshape(-1, s[-1] * s[-2]), X_test.reshape(-1, s[-1] * s[-2])

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"Y_train: {Y_train.shape}, Y_test: {Y_test.shape}")
print(f"Y_train_onehot: {Y_train_onehot.shape}, Y_test_onehot: {Y_test_onehot.shape}")
print(f"X_train_flat: {X_train_flat.shape}, X_test_flat: {X_test_flat.shape}")

X_train: (9514, 100, 6), X_test: (4078, 100, 6)
Y_train: (9514,), Y_test: (4078,)
Y_train_onehot: (9514, 6), Y_test_onehot: (4078, 6)
X_train_flat: (9514, 600), X_test_flat: (4078, 600)


In [5]:
y_pred_onehot = classifier.predict(X_test)
y_pred = y_pred_onehot.argmax(axis=1)

from sklearn.metrics import classification_report

print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       358
           1       1.00      0.99      0.99       373
           2       0.94      0.94      0.94      1027
           3       0.93      0.92      0.93       890
           4       0.96      0.96      0.96       431
           5       0.98      0.99      0.99       999

    accuracy                           0.96      4078
   macro avg       0.96      0.96      0.96      4078
weighted avg       0.96      0.96      0.96      4078



In [6]:
from sklearn.metrics import precision_recall_fscore_support

prec, recall, fscore, sup = precision_recall_fscore_support(Y_test, y_pred, average='macro')
print(f"Precision: {prec:.1%}, Recall: {recall:.1%}, Fscore: {fscore:.1%}")

Precision: 96.1%, Recall: 95.8%, Fscore: 95.9%


In [7]:
import pandas as pd

In [8]:
all_data.groupby("activity").count()

,subject,trial,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,...,xaccel_norm,yaccel_norm,zaccel_norm,magnitude_norm,gravity.x.real,gravity.y.real,gravity.z.real,userAcceleration.x.real,userAcceleration.y.real,userAcceleration.z.real
activity,,,,,,,,,,,,,,,,,,,,,
dws,131856,131856,131856,131856,131856,131856,131856,131856,131856,131856,...,131856,131856,131856,131856,131856,131856,131856,131856,131856,131856
jog,134231,134231,134231,134231,134231,134231,134231,134231,134231,134231,...,134231,134231,134231,134231,134231,134231,134231,134231,134231,134231
sit,338778,338778,338778,338778,338778,338778,338778,338778,338778,338778,...,338778,338778,338778,338778,338778,338778,338778,338778,338778,338778
std,306427,306427,306427,306427,306427,306427,306427,306427,306427,306427,...,306427,306427,306427,306427,306427,306427,306427,306427,306427,306427
ups,157285,157285,157285,157285,157285,157285,157285,157285,157285,157285,...,157285,157285,157285,157285,157285,157285,157285,157285,157285,157285
wlk,344288,344288,344288,344288,344288,344288,344288,344288,344288,344288,...,344288,344288,344288,344288,344288,344288,344288,344288,344288,344288


In [23]:
all_data[all_data["activity"] == "jog"][["userAcceleration.x.real", "userAcceleration.y.real", "userAcceleration.z.real"]].std().mean()

9.508515736779822

In [10]:
xx = all_data[all_data["activity"] == "sit"]["userAcceleration.x.real"]
print(xx[xx < 0].mean())
print(xx[xx > 0].mean())

-0.09261165929837795
0.11468632105985443
